# Web Chatbot that Acts Like Me!

In the folder `me` I've put a file `Profile.pdf` - it's a PDF download of my LinkedIn profile.

I've also made a file called `summary.txt` with a short description about me.

In [22]:
# read env key for openai
from dotenv import load_dotenv
# library to create OpenAI object
from openai import OpenAI
# to read pdf file and extract text
from pypdf import PdfReader
# to make the UI
import gradio as gr

In [56]:
load_dotenv(override=True)
# Create an instance of the OpenAI class
openai = OpenAI()

Read the pdf file and extract all text in a string.

In [24]:
reader = PdfReader("me/Profile.pdf")

linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)

   
Contact
windwatcher17@gmail.com
www.linkedin.com/in/ayushi-
saxena-416bb9168 (LinkedIn)
Top Skills
React.js
PostgreSQL
Django
Languages
English (Professional Working)
Honors-Awards
Third Position
Certificate of Gratitude
Participation
First Position
Certificate of Appreciation
Ayushi Saxena
Software Developer at CDSA | Data Science and Applications |
Python, Flask, MySQL, Transformers, LLM, RAG
Delhi, India
Summary
Ayushi is a purpose-driven Full Stack Developer who believes in
building technology with compassion, inclusivity, and sustainability at
its core.
She holds a degree in Data Science and Programming from IIT
Madras and brings strong experience in application development,
data-driven problem solving, and AI-driven innovation. With a
foundation in Python, Flask, Django, React, LangChain, SQL and
Vector-based databases, she has grown into a versatile developer
capable of designing, building, and deploying scalable applications.
Ayushi also works extensively in modern AI techn

In [25]:
# Read the summary text
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

Setting the prompts:-

In [26]:
name = "Ayushi Saxena"

*System Prompt*: Defines how the AI should behave overall for the task. It sets the context for the task at hand and the format or the way it should be responded to.

*User Prompt*: This is the instruction or question the user types.

System prompt always wins if there’s a conflict. For example,
* If the system prompt says “don’t give medical advice”
and the user says “give me medical advice” → refused or redirected.
* If the system prompt says “be concise” → even long user prompts get short answers.

In [27]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [28]:
system_prompt

"You are acting as Ayushi Saxena. You are answering questions on Ayushi Saxena's website, particularly questions related to Ayushi Saxena's career, background, skills and experience. Your responsibility is to represent Ayushi Saxena for interactions on the website as faithfully as possible. You are given a summary of Ayushi Saxena's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Ayushi Saxena. I am a software engineer and GenAI expert. I'm from Delhi, currently working in Sonipat, Haryana.\nI love online gaming. Currently I'm playing Brawl Stars but I also like strategic games like Sid Mier's Civilization series.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nwindwatcher17@gmail.com\nwww.linkedin.com/in/ayushi-\nsaxena-416bb9168 (LinkedIn)\nTop Skills\nReact.js\nPostgreSQL\nDjango\nLangua

Gradio requires a function it can callback to when it needs to process when the user types something in.

In [29]:
def chat(message, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    return response.choices[0].message.content

In [30]:
gr.ChatInterface(chat).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


<hr>

## Evaluating LLMs using other LLM

Pydantic is a Python framework for data validation and settings management.

Pydantic = schema + validation + parsing + serialization

Pydantic lets you:
* Define data schemas using type hints
* Validate and parse input data automatically
* Get clear errors instead of silent bugs
* Work cleanly with JSON, APIs, and configs

In [31]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [49]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."
evaluator_system_prompt += """

IMPORTANT:
Respond with VALID JSON ONLY.
No markdown. No explanations. No extra text.

The response MUST match this schema exactly:
{
  "is_acceptable": boolean,
  "feedback": string
}
"""

In [50]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
def build_eval_prompt(reply, message, history):
    return f"""
        {evaluator_system_prompt}

        Conversation history:
        {history}

        Latest user message:
        {message}

        Agent response:
        {reply}

        Evaluate the agent response and return JSON only.
    """

In [60]:
from google import genai
import os

gemini = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

# response = client.models.generate_content(
#     model="gemini-2.5-flash", contents="Explain how AI works in a few words"
# )
# print(response.text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [61]:
import json

def evaluate(reply, message, history) -> Evaluation:
    prompt = build_eval_prompt(reply, message, history)
    response = gemini.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )
    text = response.text.strip()
    try:
        data = json.loads(text)
        return Evaluation(**data)
    except Exception as e:
        raise ValueError(
            f"Gemini returned invalid JSON:\n{text}"
        ) from e

In [54]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content
reply

"As of now, I do not hold a patent. My focus has been more on developing applications and working with AI technologies. If you're interested in my work or projects, feel free to ask!"

In [62]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The agent accurately answers the question based on the provided context and maintains a professional and engaging tone, offering further interaction.')

In [63]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [64]:
def chat(message, history):
    # for testing purpose, failing the evaluation
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The agent's response is in Pig Latin, which is highly unprofessional and inappropriate for the instructed persona of Ayushi Saxena talking to a potential client or future employer on her website. It makes the response difficult to understand and does not align with the 'professional and engaging' instruction.
